# This transforms 'Registration_RD and Cancer 12_12_2016.xlsx' spreadsheet generated 12 December 2016 into csv for application to HES 

## import libraries and declare functions

In [ ]:
import pandas as pd
import numpy as np
import re
from matplotlib_venn import venn2
%matplotlib inline

In [ ]:
def modulo(num):   #define a function to test modulo 11 check bit
    if type(num) != int: return False
    num = str(num)
    if len(num) != 10:
        return False
    sum = 0
    for n in range(9):
        sum += int(num[n]) * (10-n)
    rem = 11 - sum%11
    if rem == 11:
        rem = 0
    elif rem == 10:
        return False
    if rem == int(num[9]):
        return True
    else:
        return False
    

In [ ]:
def genderCode(gender): #convert gender strings(male, female, '1', '2' etc) to int codes (1,2)
    if gender == 'Male':
        return 1
    elif gender == '1':
        return 1
    elif gender == 1:
        return 1
    elif gender == 'Female':
        return 2
    elif gender == '2':
        return 2
    elif gender == 2:
        return 2
    elif gender == 'Not Specified':
        return 9
    elif gender == '9':
        return 9
    elif gender == 9:
        return 9
    else:
        return 0   #default to 0 = not known for anything not in the above
    

In [ ]:
def postcodeCheck(code): #checks the postcode, returns True if it is valid, False if not
    result = re.findall("[A-Z]{1,2}[\dR][\dA-Z]? \d[ABD-HJLNP-UW-Z]{2}",str(code))
    if len(result) == 0:
        return False
    else:
        return True

In [ ]:
def postcodeFix(code): #adds a missing space to incorrect postcodes
    if len(str(code)) < 6 or len(str(code)) > 9:  #if there are not six digits return an error
        return np.NaN
    else:
        code = str(code)
        if code[-5:-3] == '  ':   #if there is a double gap
            return code[0:len(code)-4] + code[-3:]  #return a string with one gap
        elif code[-4:-3] != ' ':  #if there is no gap
            return code[0:len(code)-3] + ' ' + code[-3:]  #return a string with a gap
        else:
            return np.NaN

### load raw data sets

In [ ]:
location = 'C:/Users/Andrew Devereau/Documents/GeL/secondary data/Data applications/HES application December 2016/'
filename = 'Registration_RD and Cancer 12_12_2016.xlsx'  #there are two tabs in the spreadsheet - RD and Cancer

In [ ]:
RDData = pd.read_excel(location+filename, sheetname='RD', header=0,)  #get the rare disease data tab

In [ ]:
CaData = pd.read_excel(location+filename, sheetname='Cancer', header=0,)    #get the Cancer data tab

In [ ]:
RDData.head()

In [ ]:
RDData.rename(columns={'Participant Identifiers Id': 'Participant Id'}, inplace=True)  #rename this column to be the same as Cancer
RDData.rename(columns={'Person Stated Gender Id': 'Person Stated Gender'}, inplace=True)  #rename to be consistent with pilot data
CaData.rename(columns={'Person Stated Gender Id': 'Person Stated Gender'}, inplace=True)  #rename to be consistent with pilot data

In [ ]:
RDData.info()

In [ ]:
CaData.head()

In [ ]:
CaData.info()

In [ ]:
pilotData = pd.read_excel(location+'Pilot_Participants.xlsx', sheetname=0, header=0,)  #load pilot data, same as October application
pilotData['Date Of Birth'] = pd.to_datetime(pilotData['Date Of Birth'], errors = 'coerce')  #turn dob into datatime

In [ ]:
pilotData.info()

### Combine the data sets

In [ ]:
fullData = pd.concat([RDData, CaData, pilotData],ignore_index=True)    #use ignore_index to re-index the rows

In [ ]:
fullData = fullData[['Participant Id', 'Nhs Number', 'Date Of Birth', 'Person Stated Gender', 'Pcd Postcode']] #rearrange the column order

In [ ]:
fullData.info()

### First correct gender codes - make all into integers

In [ ]:
fullData['Person Stated Gender'].value_counts()  #see what values are held for genders - integers, strings, words and missing values

In [ ]:
fullData[fullData['Person Stated Gender'] == 'X']  #this is a new participant since October, there is little information

### Make all gender codes integers. The blank values will be given 0. The record with X will also be set to 0.

In [ ]:
fullData['Person Stated Gender'] = fullData['Person Stated Gender'].apply(genderCode)  #genderCode function replaces codes with integers

In [ ]:
fullData['Person Stated Gender'].value_counts()  #show corrected gender values

### Next correct postcodes 

In [ ]:
invalidPostcode = fullData[~fullData['Pcd Postcode'].apply(postcodeCheck)]   #find any rows with invalid postcodes using postcodeCheck function
invalidPostcode.count() #show the number of rows in the invalid postcode list. Only 1431 rows have a non-null postcode

In [ ]:
fullData.set_value(invalidPostcode.index,'Pcd Postcode', fullData.iloc[invalidPostcode.index,4].apply(postcodeFix))  #fix postcodes with incorrect spaces. Column 4 is the postcode column
fullData[~fullData['Pcd Postcode'].apply(postcodeCheck)].count()   #count how many postcodes are still invalid - nearly 1400 have been corrected

In [ ]:
fullData['Pcd Postcode'] = fullData['Pcd Postcode'].fillna(value='NULL')  #replace reamining missing postcodes with NULL

### There are many identical rows - remove these

In [ ]:
fullData = fullData.drop_duplicates().reset_index(drop=True)  #reset the index after dropping duplicate rows

In [ ]:
fullData.info()   #over 6000 duplicate rows were removed

### Look at NHS numbers

In [ ]:
fullData['Nhs Number'].value_counts()[:30]  #look for most frequent duplicated numbers

In [ ]:
invalid = fullData[~fullData['Nhs Number'].apply(modulo)]   #find any rows with invalid NHS no
invalid.to_excel(location+'invalidNHS.xlsx', index=False)   #save an excel sheet of invalid NHS numbers
invalid   #show invalids - there are 61

its possible that some without a valid NHS number could be a duplicate with a main programme participant with a different Participant Id

### set invalid NHS numbers to NULL

In [ ]:
fullData.set_value(invalid.index,'Nhs Number', 'NULL')   #set the invalid NHS No to NULL
fullData.loc[invalid.index, :]   #show the changed rows

### Remove fake NHS numbers e.g. 22222222222, 33333333333, 9999999999 

In [ ]:
fakeNHSlist = [2222222222, 3333333333, 4444444444, 5555555555, 6666666666, 7777777777, 8888888888, 9999999999]   #look for these fake numbers
fakeNHS = fullData[(fullData['Nhs Number'].isin(fakeNHSlist))]   #get a subset of records with fake numbers
fakeNHS.to_excel(location+'fakeNHSlist.xlsx', index=False)    #save the fake NHS numbers to excel

In [ ]:
fullData.set_value(fakeNHS.index, 'Nhs Number', 'NULL')   #set fake values to NULL
print(len(fakeNHS))  #there were 76 'fake' NHS numbers

### Remove invalid dates and replace with NULL

In [ ]:
tooOld = fullData[(fullData['Date Of Birth'] < '1916-01-01')]   #find dates of birth which are before 1901
tooOld

In [ ]:
tooYoung = fullData[(fullData['Date Of Birth'] > '2016-12-12')]   #find dates of birth which are after today (12/12/16)
tooYoung

In [ ]:
tooOld.to_excel(location+'tooOld.xlsx', index=False)  #save tooOld and tooYoung sets to excel
tooYoung.to_excel(location+'tooYoung.xlsx', index=False)

In [ ]:
fullData.set_value(tooOld.index,'Date Of Birth', np.NaN)   #set the invalid Dates of birth to NULL
fullData.set_value(tooYoung.index,'Date Of Birth', np.NaN)   #set the invalid NHS No to NULL
fullData.loc[tooOld.index, :]   #show the changed rows - there were 305 too old patients

In [ ]:
fullData.loc[tooYoung.index, :]   #show the changed rows - only one patient was too young

### Analyse duplicate NHS Numbers.
There are a lot of duplicate Nhs numbers with different genders, Ids, DOB or postcode. Assume that the NHS No is correct

In [ ]:
counts = fullData['Nhs Number'].value_counts()  #get a count of Nhs number frequency
counts[:20]  #show the frequency - max non-null frequency is 3

In [ ]:
dups = fullData[(fullData.duplicated(['Nhs Number'], keep=False) & (fullData['Nhs Number'] != 'NULL'))] #get a subset of duplicates
len(dups)   

First find the triples, i.e. count = 3

In [ ]:
triples = fullData[(fullData['Nhs Number'].isin(counts.index[counts > 2]) & (fullData['Nhs Number']!= 'NULL'))]

In [ ]:
triples.sort_values('Nhs Number')  #show the results

In [ ]:
len(triples)  #there are 33 triples i.e. 11 sets of 3

In [ ]:
triples.sort_values('Nhs Number').to_excel(location+'triples.xlsx', index=False)  #save triples to excel

Next find all remaining duplicates 

In [ ]:
# get two aligned subsets of the duplicates with one of each duplicate in each subset
dupsFirst = dups[(dups.duplicated(['Nhs Number'], keep ='first')) & (~dups['Nhs Number'].isin(triples['Nhs Number']))].sort_values('Nhs Number')
dupsLast = dups[(dups.duplicated(['Nhs Number'], keep ='last')) & (~dups['Nhs Number'].isin(triples['Nhs Number']))].sort_values('Nhs Number')

In [ ]:
len(dupsFirst)

In [ ]:
len(triples) + len(dupsFirst) + len(dupsLast)  #all triple and duplicate subsets lengths add up to the length of dups therefore all cases are found

In [ ]:
pd.concat([dupsFirst, dupsLast]).to_excel(location+'dups.xlsx', index= False)  #send two sets of doubles to an excel file for side-by-side analysis

### Analyse gender = 9 records
There are 1135 values of 9 which are 'not specified'. These should probably be 0, 'not known'. However we will not remove or null these to see what is returned by HES, then we wil correct them in the next application

In [ ]:
gender9 = fullData[fullData['Person Stated Gender'] == 9]  #get the participants with gender set to 9
len(gender9)

In [ ]:
venn2([set(gender9['Nhs Number']), set(dups['Nhs Number'])])   #this shows that many genders = 9 are duplicated suggesting the corrected record and original were in the dataset

In [ ]:
gender9 = fullData[fullData['Person Stated Gender'] == 9]  #extract all people with gender = 9

In [ ]:
gender9.to_excel('gender9.xlsx', index=False)   #save the gender = 9 cases to excel

### Set any remaining invalid dates to NULL

In [ ]:
fullData['Date Of Birth'] = [d.strftime('%Y-%m-%d') if not pd.isnull(d) else 'NULL' for d in fullData['Date Of Birth']] #remove NaT vaules in dates and preserve format

### Now remove some of the NHS number duplication
This is done after other fields have been set to NULL

In [ ]:
doubles = dups[~dups['Nhs Number'].isin(triples['Nhs Number'])]  #these are the double duplicates

In [ ]:
doubles.head()

Split the duplicates into two aligned sets with one duplicate row in each set

In [ ]:
doubleFirst = doubles.groupby('Nhs Number').head(1).sort_values(by='Nhs Number').reset_index()  #split into two groups and align indices
doubleLast = doubles.groupby('Nhs Number').tail(1).sort_values(by='Nhs Number').reset_index()

In [ ]:
doubleFirst.head()  #these are the first instances of each duplicate

In [ ]:
doubleLast.head()  #these are the second instances of each duplicate

Now find cases where all the duplicates are identical except for one gender which is 9 and the other is 1 or 2. The row with the 9 value is then dropped, but it can be in either the first or second set of duplicates

In [ ]:
len(fullData)  #check the length of the data set before removing rows

In [ ]:
genderChangeFirst = doubleFirst.loc[(doubleFirst['Person Stated Gender'] == 9) & (doubleLast['Person Stated Gender'].isin([1,2])) & (doubleFirst['Date Of Birth'] == doubleLast['Date Of Birth']) & (doubleFirst['Pcd Postcode'] == doubleLast['Pcd Postcode'])]

In [ ]:
fullData.drop(fullData.index[genderChangeFirst['index']], inplace=True)  #drop the selected duplicate rows in the first duplicate set

In [ ]:
genderChangeLast = doubleLast.loc[(doubleLast['Person Stated Gender'] == 9) & (doubleFirst['Person Stated Gender'].isin([1,2])) & (doubleFirst['Date Of Birth'] == doubleLast['Date Of Birth'])& (doubleFirst['Pcd Postcode'] == doubleLast['Pcd Postcode'])]

In [ ]:
fullData.drop(fullData.index[genderChangeLast['index']], inplace=True)   #drop the selected duplicate rows in the second duplicate set

In [ ]:
len(fullData)  #check the new data set length. 614 duplicates were removed

Repeat the process for cases where one duplicate has a NULL postcode but the other has a valid postcode

In [ ]:
PCChangeFirst = doubleFirst.loc[(doubleFirst['Pcd Postcode'] == 'NULL') & (doubleLast['Pcd Postcode'] != 'NULL') & (doubleFirst['Date Of Birth'] == doubleLast['Date Of Birth']) & (doubleFirst['Person Stated Gender'] == doubleLast['Person Stated Gender'])]

In [ ]:
fullData.drop(fullData.index[PCChangeFirst['index']], inplace=True)  #drop the duplicates with NULL postcodes in hthe first duplicate set

In [ ]:
PCChangeLast = doubleLast.loc[(doubleLast['Pcd Postcode'] == 'NULL') & (doubleFirst['Pcd Postcode'] != 'NULL') & (doubleFirst['Date Of Birth'] == doubleLast['Date Of Birth']) & (doubleFirst['Person Stated Gender'] == doubleLast['Person Stated Gender'])]

In [ ]:
fullData.drop(fullData.index[PCChangeLast['index']], inplace=True)  #drop the duplicates with NULL postcodes in the second duplicate set

In [ ]:
fullData = fullData.reset_index(drop=True)   #reset the full dataset index following dropping of rows

In [ ]:
len(fullData)  #re-check the full data set length. 60 more duplicates were removed

Now remove any rows in the triples which are identical to other rows except for a gender = 9 or a NULL postcode and the others in the triple have valid data. In some cases two records in the truple can be removed, in others only one. If two genders are given but one is 1 and one is 2 both records are left in. Removal is manual - i.e. those to be removed are identifed by eye and indicated in a list

In [ ]:
triples = fullData[(fullData['Nhs Number'].isin(counts.index[counts > 2]) & (fullData['Nhs Number']!= 'NULL'))]  #get this again in case the indices are different now

In [ ]:
triples.sort_values('Nhs Number')

In [ ]:
removals = [13309, 11055, 8611, 8674, 8671, 47, 11018, 9247, 10745, 12289, 10698, 10702, 11062]  #these are the records to be removed

In [ ]:
fullData.drop(fullData.index[removals], inplace=True)  #drop the triple duplicates given in the removals list

In [ ]:
fullData.info()  #the final set has 23,893 records with no missing values

## Save the final CSV file for submission to HES

In [ ]:
fullData.to_csv(location+'121216.csv', index=False, date_format = '%d/%m/%Y')   #save as CSV with 'dd/mm/yyyy' date format